# implement Gradient Descent Method

1. the focus is achieving random gradient descent method.

In [1]:
import numpy as np

In [2]:
R = np.array([[4,0,1,0,5],
     [1,2,1,3,5],
     [4,5,3,1,0],
     [2,3,0,2,5],
     [5,1,4,0,0],
     [0,3,2,4,1]])
# 为0的地方并不是评分为0，而是用户并没有对该物品进行评价。没有评分的地方并不用考虑它的误差。
R.shape

(6, 5)

In [6]:
    """_summary_
    输入
    R 是m*n的评分矩阵
    K 隐性特征向量维度。注意这个特征维度是人为定义的。但是按道理这不应该有人为定义。
    steps/max_iter 最大迭代步长
    alpha 步长
    lamda 正则化系数
    
    输出
    分解之后的P和Q
    P 初始化用户特征矩阵m*k
    Q 初始化物品特征矩阵k*n
    """
    
    # 对超参数进行赋值
    K=2
    max_iter = 5000 #迭代次数多意味着步长比较小。
    alpha = 0.0002
    lamda = 0.004
    
    def grad(R, K=2, max_iter= 5000, alpha=0.001, lamda= 0.002, cost_threshold = 0.0001):
        m = len(R)
        n = len(R[0])
        
        P = np.random.rand(m, K)
        Q = np.random.rand(K, n)
        
        for step in range(max_iter):
            # 对所有的用户u和物品i做遍历。对对应的Pu和Qi向量进行梯度下降。
            for u in range(m):
                for i in range(n):
                    # 对于每一个大于0的评分，求出评分误差。
                    if R[u][i] > 0:
                        eui = np.dot(P[u, :],Q[:, i]) - R[u,i]
                        
                        # 带入梯度下降的公式，按照梯度下降算法更新当前的Pu和Qi。也就是按照K个隐藏维度来更新。
                        for k in range(K):
                            # 注意这里和公式不同的地方在于求和公式。由于求和是对i在求和，而本计算是包含在
                            # for i in range(n):当中的，就相对于每个步骤都减去了一个对于i的元素，所以不
                            # 用再求和了。
                            P[u][k] = P[u][k] - alpha * (2 * eui * Q[k][i] - 2 * lamda * P[u][k])
                            # 同样的
                            Q[k][i] = Q[k][i] - alpha * (2 * eui * P[u][k] - 2 * lamda * Q[k][i])
                    
            # u和i遍历完成。所有特征向量都更新完成。可以计算预测评分矩阵。
            # predictR = np.dot(P, Q)
            # 计算当前的损失函数。
            cost = 0
            
            for u in range(m):
                for i in range(n):
                    # 在评分矩阵R中为0的不计算损失函数，原因依然是为0的评分可能是用户没有评分。
                    if R[u][i] > 0:
                        cost += (np.dot(P[u, :],Q[:, i]) - R[u,i]) ** 2
                        for k in range(K):
                            cost += lamda * (P[u][k] ** 2 + Q[k][i] ** 2)
            # 当损失函数小于某一个特定阈值时退出。
            if cost < cost_threshold:
                break
        return P, Q, cost

In [12]:
P, Q, cost = grad(R, K, max_iter, alpha, lamda)
predictR = np.dot(P, Q)
print("origin R is {}, \n\n predict Matrix is {}, \n\n  User matrix is {}, \n\n Item matrix is {}, \n\n Cost is {}\n\n".format(R, predictR, P, Q, cost))

origin R is [[4 0 1 0 5]
 [1 2 1 3 5]
 [4 5 3 1 0]
 [2 3 0 2 5]
 [5 1 4 0 0]
 [0 3 2 4 1]], 

 predict Matrix is [[3.54174477 1.97133881 2.38751299 5.76100169 4.67127264]
 [2.29920827 2.23295108 1.63989292 2.53191696 4.22170094]
 [3.44020532 5.25779975 2.6346376  1.35937725 8.70807992]
 [2.34440667 2.84890325 1.72613174 1.85673942 5.01839476]
 [4.99145695 1.6309815  3.25647258 9.57300412 5.15198211]
 [2.11619373 0.9429166  1.40436    3.73995738 2.49795002]], 

  User matrix is [[0.98270235 1.7975555 ]
 [1.18163621 0.73926695]
 [2.86129641 0.2461875 ]
 [1.53115451 0.49714553]
 [0.73056454 3.04805778]
 [0.45314909 1.17945485]], 

 Item matrix is [[1.08377521 1.82924256 0.8463139  0.20917847 2.95899503]
 [1.37782467 0.09665229 0.86553007 3.09055353 0.981033  ]], 

 Cost is 14.746426198168669




In [14]:
# 通过上面的结果观察，评分矩阵和实际的评分直接误差有点大。
# 这个时候思考可能分解的维度可能太低了，测试提高分解维度来观察结果。
K= 5
P, Q, cost = grad(R, K, max_iter, alpha, lamda)
predictR = np.dot(P, Q)
print("origin R is {}, \n\n predict Matrix is {}, \n\n  User matrix is {}, \n\n Item matrix is {}, \n\n Cost is {}\n\n".format(R, predictR, P, Q, cost))

origin R is [[4 0 1 0 5]
 [1 2 1 3 5]
 [4 5 3 1 0]
 [2 3 0 2 5]
 [5 1 4 0 0]
 [0 3 2 4 1]], 

 predict Matrix is [[4.00519202 3.57168317 0.9972243  1.72624065 5.00618064]
 [0.99491891 2.0010862  0.99550369 3.00899751 5.01091549]
 [4.00713658 5.01282918 3.00382068 0.99513495 5.47680465]
 [1.99631581 3.00433043 1.99866257 2.00098065 5.0086677 ]
 [5.01015864 0.99301431 4.00975012 3.13806242 5.92554697]
 [2.33711362 3.00763465 2.00458331 4.01175207 0.99424439]], 

  User matrix is [[ 0.29213849 -0.10693488  0.84047931  1.30665458  1.04771517  0.56633749
   0.58315511  0.42555361  0.65787306  0.01903948]
 [ 0.38645495  0.83553379 -0.25370681  0.87686527  0.84776242  0.87594798
  -0.07071254  1.03512286 -0.4482249   0.66191063]
 [ 0.8154881   0.33412132  0.38106457  0.70561404  0.91091142 -0.15353725
   1.16687497  0.9236073   1.40657497  0.20317671]
 [ 0.83037735  1.05063427 -0.13666737  1.0343572   0.33791416  0.30290628
   0.25888954  0.59612936  0.45406635  0.67309081]
 [-0.04771131  1.3